In [5]:
# datasets = load_dataset('conll2003')
datasets = load_dataset('csv', data_files='all_emails.csv')
#print(datasets)
from datasets import DatasetDict

# Splitting the dataset into 80% training and 20% testing
train_test_split = datasets['train'].train_test_split(test_size=0.2, seed=42)

# The resulting splits are contained in a DatasetDict object
splits = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Now you have splits['train'] and splits['test'] as your training and testing sets


print("Dataset Object Type:", type(datasets["train"]))
print("Training Examples:", len(datasets["train"]))

datasets["train"][100]

# print("Dataset Object Type:", type(datasets["train"]))
# print("Training Examples:", len(datasets["train"]))

# datasets["train"][100]

NameError: name 'load_dataset' is not defined

In [2]:
# label_list = datasets["train"].features["lebal"].feature.names
# label_list
datasets

NameError: name 'datasets' is not defined

In [ ]:
example = datasets["train"][0]
print(example)
tokenized_input = tokenizer(example["email"])
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# print(tokens)

In [ ]:
tokenized_input

In [ ]:
tokenized_input.word_ids()

In [ ]:
# tonkenize all input
def tokenizeInputs(inputs):
    
#     tokenized_inputs = tokenizer(inputs["email"], max_length = 512, truncation=True, is_split_into_words=True)
#     word_ids = tokenized_inputs.word_ids()
#     ner_tags = inputs["ner_tags"]
#     labels = [ner_tags[word_id] for word_id in word_ids]
#     tokenized_inputs["labels"] = labels
    
    tokenized_inputs = tokenizer(inputs["email"], max_length = 512, truncation=True)
    
    word_ids = tokenized_inputs.word_ids()
    label = inputs["label"]
    labels = label
    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [ ]:
example = datasets["train"][100]
print(datasets["train"][100]["label"])
tokenizeInputs(example)

In [ ]:
tokenized_datasets = datasets.map(tokenizeInputs)

In [ ]:
token_count = 0
for sample in tokenized_datasets["train"]:
    token_count = token_count + 1
    
print("Tokens in Training Set:", token_count)

In [ ]:
#tokenized_datasets = tokenized_datasets.remove_columns(["id", "tokens", "ner_tags", "pos_tags", "chunk_tags"])
tokenized_datasets = tokenized_datasets.remove_columns(["email","label"])

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(f"bigscience/{model_name}", num_labels=12).cuda()

In [ ]:
print("Parameters:", model.num_parameters())
print("Expected Input Dict:", model.main_input_name )

# Estimate FLOPS needed for one training example
sample = tokenized_datasets["train"][0]
sample["input_ids"] = torch.Tensor(sample["input_ids"])
flops_est = model.floating_point_ops(input_dict = sample, exclude_embeddings = False)

print("FLOPS needed per Training Sample:", flops_est )

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy= "epoch", # Disabled for runtime evaluation 
    evaluation_strategy="steps", #"steps", # Disabled for runtime evaluation 
    eval_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    #fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
!nvidia-smi


In [2]:
%%time

trainer.train()

NameError: name 'trainer' is not defined

In [3]:
eval_results = trainer.evaluate()
print(f"Eval Loss: {eval_results['eval_loss']}")

NameError: name 'trainer' is not defined

In [ ]:
model_tuned = BloomForTokenClassification.from_pretrained("./results/checkpoint-1172")

In [ ]:
label_names = datasets["train"].features[f"ner_tags"].feature.names

id2label = {id : label for id, label in enumerate(label_names)}
label2id = {label: id for id, label in enumerate(label_names)}

model_tuned.config.id2label = id2label
model_tuned.config.label2id = label2id

In [ ]:
model_tuned.config.id2label

In [ ]:
inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", 
    add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model_tuned(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model_tuned.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
predicted_tokens_classes